# Chatbot Demo

In [3]:
import json
from src.gen_ai.gemini import get_llm
from src.chatbot.validation.llm_validation import validation_prompt
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

prompt = PromptTemplate(template=validation_prompt,
                                       input_variables=['question',
                                                        'reference_answer',
                                                        'new_answer'])
validation_chain = LLMChain(
    llm=get_llm(),
    prompt=prompt,
    verbose=True,
)

In [13]:
response = validation_chain.invoke(
    input={'question': 'Who is the landlord (lessor)?',
                  'reference_answer': 'Heelstone Land Holdings LLC',
                  'new_answer': "''' HEELSTONE LAND HOLDINGS, LLC, a Delaware limited liability company Source: Source: 1 '''"}
)

In [24]:
from src.settings import CHATBOT_VALIDATION
import pandas as pd

site_lease_agreement = pd.read_csv(CHATBOT_VALIDATION / 'site_lease_qa.csv',  sep=';')

In [26]:
interconnection_agreement = pd.read_csv(CHATBOT_VALIDATION / 'chatbot_ia_qa.csv', sep=';')

In [33]:
interconnection_agreement[interconnection_agreement.Question.notna()]

# Initialization

In [4]:
from src.chatbot.prompt_templates import default_prompt
from src.chatbot.base import ChatbotBase
from src.chatbot.config import ChatbotConfig
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prj-ilios-ai.json"
os.environ["PROJECT_ID"] = "602280418311"
os.environ["LOCATION"] = "us-west1"
os.environ['VECTOR_STORE_INDEX_ENDPOINT_ID']='projects/602280418311/locations/us-west1/indexEndpoints/2644395833545457664'
os.environ['VECTOR_STORE_INDEX_ID']="projects/602280418311/locations/us-west1/indexes/5658473864628273152"
os.environ['GCS_VECTOR_STORE_BUCKET']="chatbot_docs"

In [2]:
chatbot = ChatbotBase(config=ChatbotConfig(vector_store_id=os.environ['VECTOR_STORE_INDEX_ENDPOINT_ID'],
                                           index_id=os.environ['VECTOR_STORE_INDEX_ID'],
                                           prompt_template=default_prompt,
                                           max_documents=10))

# Example questions

In [ ]:
## Example question
question = "Who is the tenant?"
## Run chatbot query
chatbot.invoke(prompt_input={'question': question})

In [ ]:
vector_search_response = chatbot.retriever.similarity_search(
    question, k=10
)

In [2]:
from src.vectordb.chromadb.chatbot_db import ChromaDBChatbot

In [11]:
from langchain_core.documents import Document
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
embed = VertexAIEmbeddings(
    model_name="textembedding-gecko@001", project="prj-ilios-ai", location="us-west1"
)
chromadb = Chro
chromadb = Chroma.a([Document(page_content='Who is the tenant?')], embed)
chromadb.get()
#len(chromadb.vector_store.get()['documents'])

In [9]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_core.documents import Document
# import
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks

documents = [Document(page_content="What the heck")]
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@001", project="prj-ilios-ai"
)

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = embeddings

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

In [10]:
db.get()